Ok experiment over.
Conclusion:
Zero shot still isn't within reach of 3bit falcon.
And it's too slow. Ran for a whole day and only did 3k of 2 columns.
Going to try few shot on one of the smaller models and hope it's faster.

In [1]:
import torch
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM

# If you've already downloaded the model, reference its location here:
#quantized_model_dir = "/path/to/TheBloke_WizardLM-Uncensored-Falcon-40B-3bit-GPTQ"
# Or to download it from the hub and store it in the Hugging Face cache directory:
quantized_model_dir = "TheBloke/WizardLM-Uncensored-Falcon-40B-3bit-GPTQ"

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(quantized_model_dir, use_fast=False)

model = AutoGPTQForCausalLM.from_quantized(quantized_model_dir, device="cuda:0", use_triton=False, use_safetensors=True, torch_dtype=torch.bfloat16, trust_remote_code=True)

prompt = "What is a falcon? Can I keep one as a pet?"
prompt_template = f"{prompt}\n### Response:"

tokens = tokenizer(prompt_template, return_tensors="pt").to("cuda:0").input_ids
#output = model.generate(input_ids=tokens, max_new_tokens=100, do_sample=True, temperature=0.8)
#print(tokenizer.decode(output[0]))

/home/skynet3/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
The safetensors archive passed at /home/skynet3/.cache/huggingface/hub/models--TheBloke--WizardLM-Uncensored-Falcon-40B-3bit-GPTQ/snapshots/5c2c5025f2bcbd13f76b988ce29361aac92efb43/gptq_model-3bit--1g.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.
can't get model's sequence length from model config, will set to 4096.
RWGPTQForCausalLM hasn't fused attention module yet, will skip inject fused attention.
RWGPTQForCausalLM hasn't fused mlp module yet, will skip inject fused mlp.


In [38]:
import pandas as pd
ICBeLLM_xy = pd.read_csv("/media/skynet3/8tb_a/rwd_github_private/ICBeLLM/data_out/ICBeLLM_xy.tsv", sep="\t")


In [26]:
ICBeLLM_xy.columns

Index(['crisno', 'sentence_number_int_aligned', 'crisis_text',
       'sentence_span_text', 'story_so_far', 'prompt_entity_yes_no',
       'prompt_entity_list_explicit'],
      dtype='object')

In [39]:
import os
file="/media/skynet3/8tb_a/rwd_github_private/ICBeLLM/data_out/yhat_prompt_entity_yes_no.tsv"
if os.path.exists(file):
    temp=pd.read_csv(file,sep="\t")
else:
    temp=ICBeLLM_xy[['crisno', 'sentence_number_int_aligned', 'crisis_text','sentence_span_text']].copy()
    temp.loc[:,'yhat_prompt_entity_yes_no']=None
    temp.loc[:,'yhat_prompt_entity_list_explicit']=None
    

In [40]:
#65 fails
#Grab the last 2k characters of the story to avoid oom
for i in range(ICBeLLM_xy.shape[0]):
    if pd.isna(temp['yhat_prompt_entity_yes_no'][i]):
        #prompt = "Story: " + ICBeLLM_xy['story_so_far'].fillna('').values[i][-2000:] + "\n Task: " + ICBeLLM_xy['prompt_entity_yes_no'].values[i] + "\nSentence: " + ICBeLLM_xy['sentence_span_text'].values[i] + "\n\n### Response:"
        prompt = ICBeLLM_xy['prompt_entity_yes_no'].values[i] + "\n\n" + ICBeLLM_xy['sentence_span_text'].values[i] + "\n\nAnswer:"
        #print(prompt)
        tokens = tokenizer(prompt, return_tensors="pt").to("cuda:0").input_ids
        output = model.generate(input_ids=tokens, max_new_tokens=1, do_sample=True, temperature=0.8)
        result=tokenizer.decode(output[0]).replace(prompt,'')
        #print(result)
        temp.loc[i,'yhat_prompt_entity_yes_no'] = result
        #print()

    if pd.isna(temp['yhat_prompt_entity_list_explicit'][i]):
        #prompt = "Story: " + ICBeLLM_xy['story_so_far'].fillna('').values[i][-2000:] + "\n Task: " + ICBeLLM_xy['prompt_entity_list_explicit'].values[i] + "\nSentence: " + ICBeLLM_xy['sentence_span_text'].values[i] + "\n\n### Response:"
        prompt = ICBeLLM_xy['prompt_entity_list_explicit'].values[i] + "\n\n" + ICBeLLM_xy['sentence_span_text'].values[i] + "\n\nCountries and Organizations:"
        #print(prompt)
        tokens = tokenizer(prompt, return_tensors="pt").to("cuda:0").input_ids
        output = model.generate(input_ids=tokens, max_new_tokens=20, do_sample=True, temperature=0.8)
        result=tokenizer.decode(output[0]).replace(prompt,'')
        #print(result)
        temp.loc[i,'yhat_prompt_entity_list_explicit'] = result
        
    temp.to_csv("/media/skynet3/8tb_a/rwd_github_private/ICBeLLM/data_out/yhat_prompt_entity_yes_no.tsv", sep="\t", index=False)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [36]:
  ICBeLLM_xy['prompt_entity_list_explicit']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
37738   NaN
37739   NaN
37740   NaN
37741   NaN
37742   NaN
Name: prompt_entity_list_explicit, Length: 37743, dtype: float64

In [10]:
[prompt]

['Story: Bolshevik Russia, precursor to the Soviet Union The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918. The Western Allies were anxious to maintain an eastern front against Germany and wished to prevent the Germans from seizing large stocks of arms in Vladivostok In pursuit of these goals the Allies attempted to use the 40,000 troops of the Czech. Legion then in Siberia. The Czech. Legion, an anomalous nonstate actor (NSA), refused to be disarmed by Bolshevik forces. Serious clashes between them ensued triggering a crisis for Russia in May 1918.\n Task: Given the story above, extract a comma separated list of every country or organization explicitly mentioned in the sentence below.\nSentence: Intermittent fighting lasted for almost two years: the Czech. Legion, fighting alongside the ""Whites,"" acquired control of large parts of western S

In [38]:
ICBeLLM_xy['story_so_far'].fillna('').values[i] 

''

In [24]:
pd.isna(temp['yhat_prompt_entity_yes_no'][1])

True

In [13]:


temp['yhat_prompt_entity_yes_no'][0] = "asdasdasd"


/tmp/ipykernel_1732767/3876658544.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['yhat_prompt_entity_yes_no']=None
/tmp/ipykernel_1732767/3876658544.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['yhat_prompt_entity_yes_no'][0] = "asdasdasd"
